In [1]:
import sys
import numpy as np
import pandas as pd
import cv2
import pickle

from os import listdir 
from os.path import isfile, join
from tifffile import imread, imsave
from glob import glob

np.random.seed(42)


import logging
logger = logging.getLogger()  # Logger

logger.setLevel("INFO")
#logging.info("Hello")

In [2]:
def write_pickle(path, obj):
    with open(path, "wb") as f:
        pickle.dump(obj, f)
        
def read_pickle(path):
    with open(path, "rb") as f:
        obj = pickle.load(f)
    return obj

Load data

In [56]:
print(df_labels.columns)

Index(['Cartana', 'x', 'y', 'celltypes_fine', 'celltypes_coarse', 'Cartana_x',
       'Cartana_y', 'Patient ID', 'Pseudonym'],
      dtype='object')


In [57]:
print(df_labels['celltypes_fine'].unique())
print(df_labels['celltypes_coarse'].unique())

['Schwann cells' 'myCAF_POSTN' 'Classical_KRT7' 'Classical_CEACAM'
 'Acinar cells' 'T cells' 'Beta cells' 'Basal' 'Classical_TFF1'
 'Endothelial cells' 'B cells' 'Ductal cells' 'Macrophages_M2'
 'Alpha cells' 'NK cells' 'Delta cells' 'myCAF_ACTA2' 'Dendritic cells'
 'iCAF' 'Classical_REG4' 'Macrophages_M1' 'Gamma cells']
['Schwann cells' 'Fibroblasts' 'Classical PDAC' 'Acinar cells'
 'Lymphocytes' 'Endocrine cells' 'Basal-like PDAC' 'Endothelial cells'
 'Ductal cells' 'Macrophages' 'Dendritic cells']


In [ ]:
base_path = '/mnt/DataRAID/melismail/PDAC/data'
preprocessing_path ='Preprocessing_images/'
img_path = 'images/pdac_he_warped/images'
mask_path = 'images/pdac_he_warped/masks'
sliced_path = "sliced"
filtered_path = "filtered"
scaled_path = "scaled"
tile_path = "tiles"
dim = 512
dim_path = f"{dim}x{dim}"

In [ ]:
df_labels = pd.read_csv('/mnt/DataRAID/melismail/Label/labels.csv', sep='\t')

In [ ]:
image_size_dict = {file.split('_')[0]:  cv2.imread(os.path.join(base_path, img_path, file), cv2.COLOR_BGR2RGB).shape
                       for file in [f
                                    for f in os.listdir(os.path.join(base_path, img_path))
                                    if os.path.isfile(os.path.join(base_path, img_path, f)) and ".tif" in f]
                      }

In [ ]:
def find_tile_id(img_dim: tuple(), x:float, y:float, tile_dim: int):
    if img_dim is None:
        return None
    # get point coordinates from csv
    x_coord = x//target_dim[0]
    y_coord = y//target_dim[1]

    #x and y coordinates from orig.image
    img_x = (dim[0]//224) + 1
    # compute tile
    tile = x_coord + img_x * (y_coord-1)
    return tile

In [ ]:
df_labels["tile_id"] = df_labels.apply(lambda row: find_tile_id(img_dim=img_size_dict.get(row["Pseudonym"], None),
                                                              x=row["x"], y=row["y"], tile_dim=dim),
                                       axis=1)

In [ ]:
df_labels_grouped = df_labels.groupby(by="tile_id")


In [72]:
df_xcoord=df_labels.loc[df_labels['Pseudonym'] == 'IQ5WIKH4I', 'x']
print(df_xcoord)

108087    5561.0
108088    6189.0
108089    8697.0
108090    7039.0
108091    2308.0
           ...  
116110    5902.0
116111    7974.0
116112    6331.0
116113    2361.0
116114    3236.0
Name: x, Length: 8028, dtype: float64


In [73]:
y_lst=list(df_labels.loc[df_labels['Pseudonym'] == 'IQ5WIKH4I', 'y'])
print(df_ycoord)

108087    5017.0
108088    6293.0
108089    5249.0
108090    5905.0
108091    4802.0
           ...  
116110    6301.0
116111    6079.0
116112    4313.0
116113    2621.0
116114    7313.0
Name: y, Length: 8028, dtype: float64


In [74]:
x_lst = df_xcoord.to_list()

In [75]:
print(x_lst[:10])

[5561.0, 6189.0, 8697.0, 7039.0, 2308.0, 4270.0, 6420.0, 3262.0, 8044.0, 4006.0]


In [76]:
print(len(x_lst))

8028


In [77]:
y_lst = df_ycoord.to_list()

In [30]:
print(y_lst[:10])

[2739.0, 2415.0, 8015.0, 7031.0, 7726.0, 5032.0, 1391.0, 1969.0, 9337.0, 3232.0]


In [78]:
print(len(y_lst))

8028


Process data

In [79]:
img = cv2.imread("/mnt/DataRAID/melismail/Preprocessing/data/images/IQ5WIKH4I_rgb_warped.tif", cv2.COLOR_BGR2RGB)
dim = img.shape
target_dim = (224, 224)
tile_lst = []

#for img in enumerate(img_list):
for x,y in zip(x_lst, y_lst):
    #print(x)
    #print(y)
    
    # get point coordinates from csv
    x_coord = x//target_dim[0]
    #print(x_coord)
    y_coord = y//target_dim[1]
    #print(y_coord)
                
    #x and y coordinates from orig.image
    img_x = (dim[0]//224) + 1
    #img_y = (dim[1]//512) + 1
    #print(img_x)
    #print(img_y)
                
    # compute tile
    tile = x_coord + img_x * (y_coord-1)
    #print(tile) 
    
    tile_lst.append(tile)

In [80]:
print(len(tile_lst))
print(tile_lst[:10])

8028
[822.0, 1053.0, 874.0, 981.0, 770.0, 703.0, 1130.0, 850.0, 643.0, 473.0]


In [81]:
df_celltypes = df_labels.loc[df_labels['Pseudonym'] == 'IQ5WIKH4I', 'celltypes_fine']
print(df_celltypes)

108087           Beta cells
108088          myCAF_POSTN
108089              B cells
108090              T cells
108091       Macrophages_M2
                ...        
116110          myCAF_ACTA2
116111    Endothelial cells
116112         Ductal cells
116113          myCAF_ACTA2
116114       Classical_KRT7
Name: celltypes_fine, Length: 8028, dtype: object


In [82]:
celltypes_lst = df_celltypes.to_list()

In [83]:
df_first = pd.DataFrame(tile_lst, columns=['tiles'])
df_first['Pseudonym'] = np.nan
df_first['Pseudonym'] = df_first['Pseudonym'].fillna('IQ5WIKH4I')
df_first["PS"] = 'IQ5WIKH4I'
print(df_first)

       tiles  Pseudonym
0      822.0  IQ5WIKH4I
1     1053.0  IQ5WIKH4I
2      874.0  IQ5WIKH4I
3      981.0  IQ5WIKH4I
4      770.0  IQ5WIKH4I
...      ...        ...
8023  1052.0  IQ5WIKH4I
8024  1023.0  IQ5WIKH4I
8025   712.0  IQ5WIKH4I
8026   390.0  IQ5WIKH4I
8027  1192.0  IQ5WIKH4I

[8028 rows x 2 columns]


In [84]:
df_first['celltypes'] = celltypes_lst 
print(df_first)

       tiles  Pseudonym          celltypes
0      822.0  IQ5WIKH4I         Beta cells
1     1053.0  IQ5WIKH4I        myCAF_POSTN
2      874.0  IQ5WIKH4I            B cells
3      981.0  IQ5WIKH4I            T cells
4      770.0  IQ5WIKH4I     Macrophages_M2
...      ...        ...                ...
8023  1052.0  IQ5WIKH4I        myCAF_ACTA2
8024  1023.0  IQ5WIKH4I  Endothelial cells
8025   712.0  IQ5WIKH4I       Ductal cells
8026   390.0  IQ5WIKH4I        myCAF_ACTA2
8027  1192.0  IQ5WIKH4I     Classical_KRT7

[8028 rows x 3 columns]


In [85]:
write_pickle(path='/mnt/DataRAID/melismail/Preprocessing/data/pickle_data/Preprocessing_lbl_tile/224x224/tilecells_tenth.pkl', obj= df_first)